In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=1
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-3
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFNSD%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag, init_values=T_bot)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FNSD%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        fluxF.backup()
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        tempS.curr[:] = tempF.curr[f2s]
        
        solverS.update_temperature_interface(interface_tag, tempS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        fluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        fluxF.curr[:] = fluxS.curr[s2f]
        
        # update residual
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=fluxF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to solid then restore
            under_relax.determine_omega(fluxF)
            under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=fluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=6.
step=1, pc_iterations=5.
step=2, pc_iterations=5.
step=3, pc_iterations=4.
step=4, pc_iterations=4.
step=5, pc_iterations=3.
step=6, pc_iterations=3.
step=7, pc_iterations=3.
step=8, pc_iterations=3.
step=9, pc_iterations=3.
step=10, pc_iterations=3.
step=11, pc_iterations=3.
step=12, pc_iterations=3.
step=13, pc_iterations=3.
step=14, pc_iterations=3.
step=15, pc_iterations=3.
step=16, pc_iterations=3.
step=17, pc_iterations=3.
step=18, pc_iterations=3.
step=19, pc_iterations=3.
step=20, pc_iterations=3.
step=21, pc_iterations=3.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=3.
step=27, pc_iterations=3.
step=28, pc_iterations=3.
step=29, pc_iterations=3.
step=30, pc_iterations=3.
step=31, pc_iterations=3.
step=32, pc_iterations=3.
step=33, pc_iterations=3.
step=34, pc_iterations=3.
step=35, pc_iterations=3.
step=36, pc_iterations=3.
step=37, pc_iterations=3.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFNSD%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.25961829 0.27919625 0.29445315 0.30553241 0.31494077 0.32327767
 0.33065441 0.33727837 0.34339015 0.34887391 0.35390461 0.35851477
 0.36282076 0.36677618 0.3704447  0.37382548 0.37698791 0.37991882
 0.38262498 0.38515755 0.38749903 0.38967779 0.39170592 0.39357912
 0.39532572 0.39693645 0.39844569 0.39983084 0.40110484 0.40229312
 0.40339587 0.40440877 0.405348   0.40620959 0.40700753 0.40773664
 0.40840984 0.40902222 0.40959231 0.4101096  0.41058012 0.41100481
 0.41139166 0.41174321 0.41206013 0.41234368 0.41259708 0.4128202
 0.41301351 0.413184   0.41332607 0.41344512 0.41353928 0.41361081
 0.41365954 0.41368796 0.4136923  0.41367419 0.41363431 0.41356989
 0.41348009 0.41336997 0.4132309  0.41306583 0.41286954 0.41264626
 0.41238704 0.41209467 0.41176142 0.41138993 0.41097904 0.41051579
 0.40999573 0.40942952 0.40879456 0.40809417 0.4073127  0.40645574
 0.40550121 0.40445211 0.40328631 0.40200079 0.40058496 0.39902542
 0.39729145 0.39535968 0.39324465 0.39087425 0.3882246  0.38528